Runing the following to install microdf in colab:

In [ ]:
# Install microdf
# !pip install git+https://github.com/PSLmodels/microdf.git

  Cloning https://github.com/PSLmodels/microdf.git to /tmp/pip-req-build-_zt74wzk
  Running command git clone -q https://github.com/PSLmodels/microdf.git /tmp/pip-req-build-_zt74wzk


In [ ]:
import pandas as pd
import numpy as np
import microdf as mdf
import plotly.express as px

In [ ]:
person = pd.read_stata(
    "https://www2.census.gov/programs-surveys/supplemental-poverty-measure/datasets/spm/spm_2019_pu.dta",
    columns=[
        "serialno",
        "sporder",
        "wt",
        "age",
        "spm_id",
        "spm_povthreshold",
        "spm_resources",
        "st",
        "puma"
    ],
)

In [ ]:
# Cleanup
person.columns = person.columns.str.lower()
person = person.rename(columns={'serialno': 'serial', 'sporder':'pernum'})

In [ ]:
person = person.astype({"serial":'int', "pernum":'int',
                          "wt":'int', "age":'int',
                          "spm_id":'int', "spm_povthreshold":'int',
                          "spm_resources": "int"}) 

In [ ]:
# Sort to just Maryland
person = person[person['st'] == 24]

In [ ]:
# assign random district
person['district'] = np.random.randint(1, 48, person.shape[0])

In [ ]:
# Assign random county
person['county'] = np.random.randint(1, 25, person.shape[0])

In [ ]:
# Replace NIUs
person = person.replace(9999999,0)

In [ ]:
# Define age groups
person['child'] = person.age < 18
person['young_child'] = person.age < 5
person['baby'] = person.age == 0

In [ ]:
# Use groupby to calculate total babies, young children, and children in each spm unit
spmu = person.groupby(['spm_id'])[['child', 'young_child', 'baby']].sum()
spmu.columns = ['spm_children', 'spm_young_children', 'spm_babies']
# merge back onto the person dataframe
person = person.merge(spmu, left_on =['spm_id'], right_index=True)

In [ ]:
# Consider three reforms
#1 a $100 universal child allowance (0-17)
#2 a $100 universal young child allowance (0-4)
#3 a $1,000 baby bonus given upon the birth of a child

def pov(reform, county):
  if county == 'Maryland':
      tp = person.copy(deep=True) 
  else:
    tp = person[person.county==county].copy(deep=True)

  if reform == 'All Children':
    tp['total_ca'] = tp.spm_children * 100 * 12
  
  if reform == 'Young Children':
    tp['total_ca'] = tp.spm_young_children * 100 * 12
  
  if reform == 'Babies':
    tp['total_ca'] = tp.spm_babies * 1_000

  tp['new_resources'] = tp.total_ca + tp.spm_resources	
  tp['still_poor'] = tp.new_resources < tp.spm_povthreshold

  # populations
  population = (tp.wt).sum()
  child_population = (tp.child * tp.wt).sum()
  young_child_population = (tp.young_child * tp.wt).sum()
  baby_population = (tp.baby * tp.wt).sum()

  # orginal poverty rates
  tp['poor'] = tp.spm_resources < tp.spm_povthreshold

  total_poor = (tp.poor * tp.wt).sum()
  total_pov_rate = (total_poor / population)

  total_child_poor = (tp.child * tp.poor * tp.wt).sum()
  child_pov_rate = (total_child_poor / child_population)

  total_young_child_poor = (tp.young_child * tp.poor * tp.wt).sum()
  young_child_pov_rate = (total_young_child_poor / young_child_population)

  total_baby_poor = (tp.baby * tp.poor * tp.wt).sum()
  baby_pov_rate = (total_baby_poor / baby_population)

  # new poverty rates
  new_total_poor = (tp.still_poor * tp.wt).sum()
  new_total_pov_rate = (new_total_poor / population)

  new_total_child_poor = (tp.child * tp.still_poor * tp.wt).sum()
  new_child_pov_rate = (new_total_child_poor / child_population)

  new_total_young_child_poor = (tp.young_child * tp.still_poor * tp.wt).sum()
  new_young_child_pov_rate = (new_total_young_child_poor / young_child_population)

  new_total_baby_poor = (tp.baby * tp.still_poor * tp.wt).sum()
  new_baby_pov_rate = (new_total_baby_poor / baby_population)

  # percent change
  total_pov_change = ((new_total_poor - total_poor) / (total_poor) * 100).round(1)
  child_pov_change = ((new_total_child_poor - total_child_poor) / (total_child_poor) * 100).round(1)
  young_child_pov_change = ((new_total_young_child_poor - total_young_child_poor) / (total_young_child_poor) * 100).round(1)
  baby_pov_change = ((new_total_baby_poor - total_baby_poor) / (total_baby_poor) * 100).round(1)
  
  return pd.Series([total_pov_change,
          child_pov_change,
          young_child_pov_change,
          baby_pov_change,
          population,
          child_population,
          young_child_population,
          baby_population,
          total_pov_rate,
          child_pov_rate,
          young_child_pov_rate,
          baby_pov_rate,
          new_total_pov_rate,
          new_child_pov_rate,
          new_young_child_pov_rate,
          new_baby_pov_rate,
                    ])

In [ ]:
counties = person.county.unique().tolist()
summary = mdf.cartesian_product({
                       'reform':['All Children', 'Young Children', 'Babies'],
                       'countyfip': ['Maryland'] + counties})

In [ ]:
def pov_row(row):
  return pov(row.reform, row.countyfip)

In [ ]:
summary[['total_pov_change',
          'child_pov_change',
          'young_child_pov_change',
          'baby_pov_change',
         'population',
         'child_population',
         'young_child_population',
         'baby_population',
          'total_pov_rate',
          'child_pov_rate',
          'young_child_pov_rate',
          'baby_pov_rate',
          'new_total_pov_rate',
          'new_child_pov_rate',
          'new_young_child_pov_rate',
          'new_baby_pov_rate',]] = summary.apply(pov_row, axis=1)

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
summary.to_csv('skeleton_county_data.csv')

In [ ]:
summary

,reform,countyfip,total_pov_change,child_pov_change,young_child_pov_change,baby_pov_change,population,child_population,young_child_population,baby_population,total_pov_rate,child_pov_rate,young_child_pov_rate,baby_pov_rate,new_total_pov_rate,new_child_pov_rate,new_young_child_pov_rate,new_baby_pov_rate
0,All Children,Maryland,-9.6,-20.4,-20.3,-19.6,5905181.0,1328582.0,356194.0,64250.0,0.129266,0.148904,0.159096,0.179984,0.116842,0.118525,0.126726,0.144638
1,All Children,16,-3.8,-2.3,-1.9,-20.4,241795.0,55522.0,17032.0,1734.0,0.123683,0.120853,0.171618,0.158593,0.119014,0.118133,0.168330,0.126298
2,All Children,2,-8.4,-16.5,-22.2,0.0,252970.0,58190.0,15350.0,4178.0,0.127173,0.150369,0.211857,0.279560,0.116437,0.125520,0.164756,0.279560
3,All Children,5,-10.9,-27.0,-52.1,-40.0,252829.0,60602.0,16468.0,3955.0,0.122197,0.162585,0.170330,0.280405,0.108860,0.118692,0.081613,0.168142
4,All Children,3,-8.7,-28.6,-12.8,-73.3,243572.0,55791.0,14931.0,2508.0,0.141157,0.128031,0.102203,0.106061,0.128886,0.091413,0.089143,0.028309
5,All Children,15,-6.9,-18.3,-4.7,0.0,237065.0,49164.0,12936.0,2647.0,0.119237,0.130400,0.192254,0.257650,0.111050,0.106582,0.183210,0.257650
6,All Children,9,-5.9,-8.9,-12.0,-50.0,244821.0,59964.0,19248.0,3261.0,0.132909,0.174905,0.226257,0.105489,0.125132,0.159362,0.199034,0.052745
7,All Children,21,-9.0,-12.8,-67.3,-100.0,236932.0,51256.0,11999.0,2773.0,0.130801,0.155104,0.087841,0.037505,0.119005,0.135184,0.028752,0.000000
8,All Children,8,-10.5,-22.8,-12.4,0.0,252738.0,51015.0,11450.0,1961.0,0.123246,0.137685,0.115546,0.192249,0.110312,0.106322,0.101223,0.192249
9,All Children,14,-12.8,-24.0,-28.8,0.0,252882.0,54846.0,14318.0,1672.0,0.125572,0.164971,0.131303,0.024522,0.109482,0.125351,0.093449,0.024522
